# Crawling ETDA (old Thai CERT encyclopedia)
- They don't have an API
- Search for APT 28: ```https://apt.etda.or.th/cgi-bin/listgroups.cgi?c=&v=&s=&m=&x=apt28``` -> list of actors
- Display APT 28: ```https://apt.etda.or.th/cgi-bin/showcard.cgi?g=Sofacy%2C%20APT%2028%2C%20Fancy%20Bear%2C%20Sednit&o=j```
- Retrieve APT 28 .json```https://apt.etda.or.th/cgi-bin/getcard.cgi?g=Sofacy%2C%20APT%2028%2C%20Fancy%20Bear%2C%20Sednit&o=j```


In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [2]:
import cfscrape
from lxml import etree

## Generic Search will list "APT groups" and "tools"

In [ ]:
search = "apt28"

In [3]:
url="https://apt.etda.or.th/cgi-bin/listgroups.cgi?c=&v=&s=&m=&x="+ search

header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
          'Accept-Encoding': 'gzip, deflate, sdch',
          'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
          'Cache-Control' : 'max-age=0',
          'Connection': 'keep-alive',
          'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.81 Safari/537.36'}

scraper = cfscrape.create_scraper()
scraped_html=scraper.get(url,headers=header).content
# print(scraped_html.decode("utf-8"))

html = etree.HTML(scraped_html)

### Retrieving all URLs of "all groups whose card contains the string" \<SEARCH_WORD>

In [42]:
elements = html.xpath("//a")

urls = []
for element in elements:
    try:
        if "showcard" in element.xpath("@href")[0]:
            urls.append("https://apt.etda.or.th"+element.xpath("@href")[0])
    except:
        next
        
urls

In [145]:
urls

['https://apt.etda.or.th/cgi-bin/showcard.cgi?g=APT%2029%2C%20Cozy%20Bear%2C%20The%20Dukes',
 'https://apt.etda.or.th/cgi-bin/showcard.cgi?g=Cyber%20Caliphate%20Army%20%28CCA%29%2C%20United%20Cyber%20Caliphate%20%28UCC%29',
 'https://apt.etda.or.th/cgi-bin/showcard.cgi?g=Hades',
 'https://apt.etda.or.th/cgi-bin/showcard.cgi?g=Snake%20Wine',
 'https://apt.etda.or.th/cgi-bin/showcard.cgi?g=Sofacy%2C%20APT%2028%2C%20Fancy%20Bear%2C%20Sednit']

## NOTE: the above list is not optimal because it doesn't point direct to one single actor. Not also that the links above are "showcard.cgi?" and to retrieve the .json file of an actor you need to change it to "getcard.cgi?" and append "&o=j"

## Creating an actor URL

In [150]:
actor_url = urls[4].replace("showcard","getcard")+"&o=j"
actor_url

'https://apt.etda.or.th/cgi-bin/getcard.cgi?g=Sofacy%2C%20APT%2028%2C%20Fancy%20Bear%2C%20Sednit&o=j'

## Retrieving an actor data

In [151]:
req = urllib.request.Request(actor_url)
response = urllib.request.urlopen(req)
jsonResponse = json.loads(response.read())
df = pd.json_normalize(jsonResponse['values'])

In [152]:
df

actor  \
0  Sofacy, APT 28, Fancy Bear, Sednit   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          names  \
0  [{'name': 'Sofacy', 'name-giver': 'Kaspersky'}, {'name': 'APT 28', 'name-giver': 'Mandiant'}, {'name': 'Fancy Bear', 'name-giver': 'CrowdStrike'}, {'name': 'Sednit', 'name-giver': 'ESET'}, {'name': 'Group 74', 'name-giver': 'Talos'}, {'name': 'TG-4127', 'name-giver': 'SecureWorks'}, {'name': 'Pawn Storm', 'name-giver': 'Trend Micro'}, {'name': 'Tsar Team', 'name-giver': 'iSight'}, {'name': 'Strontium', 'name-giver': 'Microsoft'}, {'name': 'Swallowtail', 'name-giver': 'Symantec'}, {'name': 'SIG40', 'name-giver': 'NSA'}, {'name': 'Snakemackerel', 'name-giver': 'iDefense'}, {'name': 'Iron Twilight', 'name-giver': 'SecureWorks'}, {'name': 'ATK 5', 'name-giver': 'Thales'}, {'name': 'T-APT-12', 'name-giver': 'Tencent'}, {'name': 'ITG05', 'name-giver': 'IBM'}, {'name': 'TAG-0700', 'name-giver': 'Recorded Future'}, {'name': 'UAC-0028', 'name-giver': 'CERT-UA'}, {'name': 'Grizzly Steppe', 'name-giver': 'US Government'}]   

    country  \
0  [Russia]   

                                                             sponsor  \
0  State-sponsored, two GRU units known as Unit 26165 and Unit 74455   

                          motivation first-seen  \
0  [Information theft and espionage]       2004   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     description  \
0  APT 28 is a threat group that has been attributed to Russia’s Main Intelligence Directorate of the Russian General Staff by a July 2018 U.S. Department of Justice indictment. This group reportedly compromised the Hillary Clinton campaign, the Democratic National Committee, and the Democratic Congressional Campaign Committee in 2016 in an attempt to interfere with the U.S. presidential election. APT 28 has been active since at least January 2007.\n\n(FireEye) APT28 likely seeks to collect intelligence about Georgia’